## Import core packages

In [0]:
# Import core packages
from mosaic_agent_framework import Agent, AgentToolkit
from langgraph.graph import StateGraph, END
from langgraph.graph.schema import State, RunnableConfig

# Genie and Claude helper import statements may differ in your actual SDK
from mosaic_agent_framework.agents.genie import GenieAgent
from mosaic_agent_framework.agents.foundation import FoundationModelAgent

In [0]:
# 1. Define the Genie Agent (Kaggle space)
# Assume GenieAgent automatically wires up to the desired space when configured appropriately.
genie_kaggle_agent = GenieAgent(
    genie_space="Kaggle Workd Wide Importers",  # Genie space name
    api_key="<GENIE_API_KEY>"
)

# 2. Define the Claude Agent (General Purpose + Summarizer)
# FoundationModelAgent is a generic interface for LLMs via Foundation Model APIs
claude_agent = FoundationModelAgent(
    model="anthropic.claude-3-haiku",  # change to correct model name if you want opus/sonnet/other
    api_key="<FOUNDATION_MODEL_API_KEY>"
)

In [0]:
# 3. Define The Router Agent Logic

def route_question(state: State) -> str:
    user_input = state["input"] if "input" in state else ""
    genie_keywords = ["kaggle", "notebook", "dataset", "competition", "genie"]
    if any(word in user_input.lower() for word in genie_keywords):
        return "genie"
    else:
        return "claude"

In [0]:
# 4. Define Summarization Node (calls Claude to summarize Genie answer)

def summarize_with_claude(state: State, config: RunnableConfig = None) -> State:
    genie_answer = state["genie_answer"]
    summary_prompt = f"Summarize the following response for a user:\n\n{genie_answer}"
    summary = claude_agent(prompt=summary_prompt)
    new_state = state.copy()
    new_state["output"] = summary
    return new_state

In [0]:
# 5. Set Up The LangGraph Graph

graph = StateGraph()

# Node: entrypoint (input)
graph.add_node("entry", lambda state, _: state)  # no-op input passthrough

# Node: genie_kaggle_agent
def call_genie(state: State, config: RunnableConfig = None) -> State:
    genie_response = genie_kaggle_agent(prompt=state["input"])
    state = state.copy()
    state["genie_answer"] = genie_response
    return state

graph.add_node("genie", call_genie)

# Node: claude (general-purpose)
def call_claude(state: State, config: RunnableConfig = None) -> State:
    response = claude_agent(prompt=state["input"])
    state = state.copy()
    state["output"] = response
    return state

graph.add_node("claude", call_claude)
graph.add_node("summarize", summarize_with_claude)

# Add routing condition
graph.add_edge("entry", route_question)

# Genie branch: run Genie agent, then summarize with Claude
graph.add_edge("genie", "summarize")
graph.add_edge("summarize", END)

# Claude branch: general-purpose, goes direct to END
graph.add_edge("claude", END)

In [0]:
# 6. Compile and Run

graph.set_entry_point("entry")# Example run
workflow = graph.compile()

# User input: Genie question
state = {"input": "Which are the top trending datasets on Kaggle this week?"}
result = workflow.invoke(state)
print("\nGenie Q&A (summarized via Claude):\n", result["output"])

# User input: General question (goes to Claude)
state2 = {"input": "Explain the difference between supervised and unsupervised learning."}
result2 = workflow.invoke(state2)
print("\nGeneral question (Claude):\n", result2["output"])